In [1]:
from datasketch import MinHash,MinHashLSH,MinHashLSHForest
from sklearn.feature_extraction.text import TfidfVectorizer
import jieba.posseg as pseg
import re

#读取文件
f = open('./sentences.txt','r',encoding='UTF-8')
text = f.read()
# 以句号，叹号，问号作为分隔，去掉\n换行符号
sentence = re.split('[!?。]',text.replace('\n',''))
print(sentence)

['11月10日深夜到11日凌晨，杭州阿里巴巴园区光柱擎天、灯火辉煌、不眠不休', '这是第11年双11，一个稀疏平常的日子凭空成为全民购物狂欢本就不可思议，而更不可思议的是不断刷新的消费数据', '仅用时96秒，2019天猫双11总成交额便超过100亿元，这个速度，比2018年缩短了29秒', '随后，500亿元、1000亿元，每一个成交额小目标都以更快的速度实现', '截至中证君发稿，2019天猫双11前三小时总成交额达到1325.7亿元，已经超过2018年天猫双11全天2135亿元总成交额的62%', '2018年，阿里巴巴用15小时49分钟39秒时间，打破了2017年1682亿元的双11全天成交纪录，最终以27%的增速定格在2135亿元', '今年双11，阿里巴巴又将以怎样的增速、多长时间来打破自己的纪录，它能否一举突破3000亿元大关，同时提振全国内需？作为双11购物狂欢盛典的鼻祖，阿里巴巴在过去三年双11全网成交额中占据三分之二天下，京东、苏宁以及近期势头凶猛的拼多多均处于“后来”但尚未“居上”状态', '首先来看阿里巴巴（天猫）战报', '据介绍，今年，超过1000万款商品登陆天猫双11，超100万款新品在天猫双11首发，来自78个国家和地区的跨境电商进口商品、全国1000个数字农业基地的高品质农产品、2000个传统制造业产业带的工厂直供好货，以新供给满足消费者的多元化需求', '00:01:36，2019天猫双11总成交额超100亿元', '2018年用时2分05秒，今年缩短29秒', '00:12:49，2019天猫双11总成交额超500亿元', '2018年用时26分3秒，今年缩短一半！01:03:59，2019天猫双11总成交额超1000亿元', '比2018年快了43分钟27秒，比2017年快了将近8小时', '']


In [2]:
# 最后一行如果为空，则删除
if sentence[-1]=="":
    sentence.pop()
print(sentence)

['11月10日深夜到11日凌晨，杭州阿里巴巴园区光柱擎天、灯火辉煌、不眠不休', '这是第11年双11，一个稀疏平常的日子凭空成为全民购物狂欢本就不可思议，而更不可思议的是不断刷新的消费数据', '仅用时96秒，2019天猫双11总成交额便超过100亿元，这个速度，比2018年缩短了29秒', '随后，500亿元、1000亿元，每一个成交额小目标都以更快的速度实现', '截至中证君发稿，2019天猫双11前三小时总成交额达到1325.7亿元，已经超过2018年天猫双11全天2135亿元总成交额的62%', '2018年，阿里巴巴用15小时49分钟39秒时间，打破了2017年1682亿元的双11全天成交纪录，最终以27%的增速定格在2135亿元', '今年双11，阿里巴巴又将以怎样的增速、多长时间来打破自己的纪录，它能否一举突破3000亿元大关，同时提振全国内需？作为双11购物狂欢盛典的鼻祖，阿里巴巴在过去三年双11全网成交额中占据三分之二天下，京东、苏宁以及近期势头凶猛的拼多多均处于“后来”但尚未“居上”状态', '首先来看阿里巴巴（天猫）战报', '据介绍，今年，超过1000万款商品登陆天猫双11，超100万款新品在天猫双11首发，来自78个国家和地区的跨境电商进口商品、全国1000个数字农业基地的高品质农产品、2000个传统制造业产业带的工厂直供好货，以新供给满足消费者的多元化需求', '00:01:36，2019天猫双11总成交额超100亿元', '2018年用时2分05秒，今年缩短29秒', '00:12:49，2019天猫双11总成交额超500亿元', '2018年用时26分3秒，今年缩短一半！01:03:59，2019天猫双11总成交额超1000亿元', '比2018年快了43分钟27秒，比2017年快了将近8小时']


In [35]:
# 将item_text进行分词
def get_item_str(item_text):
    item_str=""
    item = (pseg.cut(item_text))
    for i in list(item):
        #去掉停用词
        if i.word not in list(stop):
            item_str += i.word
            #tfidf_vectorizer.fit_transform的输入需要空格分隔的单词
            item_str += " "
    return item_str

#对item_str创建MinHash
def get_minhash(item_str):
    temp = MinHash()
    for d in item_str:
        temp.update(d.encode('utf8'))
    return temp
  
#设置停用词
stop = [line.strip().encode('utf-8').decode('utf-8') for line in open('stopwords.txt',encoding='utf-8',errors='ignore').readlines()]
#得到分词的document
documents = []
for item_text in sentence:
    #将item_text进行分词
    item_str = get_item_str(item_text)
    documents.append(item_str)

#创建LSH Forest和MinHash对象
minhash_list = []
forest = MinHashLSHForest()
for i in range(len(documents)):
    #得到Train_document[i]的Minhash
    temp = get_minhash(documents[i])
    minhash_list.append(temp)
    forest.add(i,temp)
#index所有key，以便可以进行检索
forest.index()

query = '01:03:59，2019天猫双11总成交额超1000亿元。'
#将item_text进行分词
item_str = get_item_str(query)
#得到item_str的minhsh
minhash_query = get_minhash(item_str)

#查询forest中与m1相似的topk个邻居
result = forest.query(minhash_query,3)
for i in range(len(result)):
    print(result[i],minhash_query.jaccard(minhash_list[result[i]]),documents[result[i]].replace(' ',''))
print('Top 3 邻居',result)

9 0.9375 00:01:362019天猫双11总成交额超100亿元
11 0.875 00:12:492019天猫双11总成交额超500亿元
12 0.578125 2018年时26分秒今年缩短一半01:03:592019天猫双11总成交额超1000亿元
Top 3 邻居 [9, 11, 12]
